In [1]:
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

plotly.tools.set_credentials_file(username='', api_key='')

## Creating a DataFrame
Often we are going to be importing data in a CSV format that conforms to some schema. We would like the ability to keep the dimensions and feature headers of that schema. Spark 1.6 has finally included the magic that is Pandas' Dataframes. These are essentially a database table but have built in operations (easily usable with numpy), dataflow optimizations, and easy data exploration.

#### SQLContext
[SQLContext](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) is the main entry point for Spark SQL functionality. A SQLContext can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files. The SQLContext takes a [SparkContext](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext) as a parameter and is in a manner of speaking a child of SparkContext.
```python
train_data_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferSchema='true').load('/FileStore/tables/libwrcg41471805568157/train.csv')
```
This code generates a dataframe by generating a SQLContext. We pass the context some options that inform the Context a header row exists in our CSV and to automatically infer our datatypes. We then load these files from a file I have uploaded to my Databricks Community Edition workspace.

Let's [check the schema](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html?highlight=printschema#pyspark.sql.DataFrame.printSchema) it has inferred and see if we agree with Spark's assumptions.

In [3]:
train_data_df = (sqlContext
                 .read
                 .format('com.databricks.spark.csv')
                 .options(header='true', inferSchema='true')
                 .load('/FileStore/tables/libwrcg41471805568157/train.csv'))
train_data_df.cache()
train_data_df.printSchema()

## What Happens Now?
There are several steps in the workflow for creating information out of data. Remember that data and information are not synonymous. Information is data in a comprehensible context. Big Data is by definition incomprehensible at a human level. It is not information that we can consume until that data is processed, cleaned, analyized, and visualized. Our understanding comes from regressions, models, statistics, and charting.

Currently we are dealing with raw data. Let's do some exploration on our new Dataframe to see what we are dealing with.

#### Data Exploration
This is the first step we want to take before we start doing any pre-processing.

Why is that?

Data exploration will allow us to ask the questions of what steps we want to take to clean our data and to normalize our data. Maybe some features are missing in a few data objects. Maybe a date in one object is in a different format than in another object (i.e. 2016/08/22 vs August 22nd, 2016). These are serious issues that we need to address so that when we start doing some analysis, we can compare values to one-another. This process in particular is called Entity Resolution.

In Databricks notebook, all we need to do is call `display(dataframe)`. Super easy, right?
This will show us by default a table printout of a prefix of our full dataframe. You can also do other data exploration with the graphing feature.

<img src="http://i.imgur.com/zxuRWlbl.png?2" alt="Drawing" />

In [5]:
train_data_df.describe().show()

In [6]:
display(train_data_df)

#### Data Cleaning
What is something that you noticed that could cause issues?
Hopefully, you noticed that some data objects (objects here are rows which represent a passenger) have a null age. This is going to cause some issues. How can we fix these non-values without affecting any population properties of the data? Let's use some [**domain knowledge**](http://www.simafore.com/blog/the-value-of-domain-knowledge-in-data-science) to do some more data exploration. Let's see if age follows any trends based off of class and/or ticket fare.

In [8]:
explore_age_df = train_data_df.orderBy('Age', ascending=True).cache()

avg_age_df = explore_age_df.where(explore_age_df['Age'].isNotNull()).groupBy('Pclass').avg('Age')
display(avg_age_df)

###### Average Age vs. Class
We've barely done any exploration, and we can already possibly see a correlation between the average age of the passenger and their passenger class. We could easily use these averages to fill in missing data or we can keep looking for greater identifiers to try to eliminate bias.

In [10]:
null_age_df = explore_age_df.where(explore_age_df['Age'].isNull()).cache()
null_age_df.describe().show()

In [11]:
p_class_df = (null_age_df
              .select('Pclass')
              .distinct()
              .collect())
p_class_count_df = (null_age_df
                    .groupBy('Pclass')
                    .count()
                    .select('count')
                    .collect())
p_class_non_null_df = (explore_age_df
                       .where(explore_age_df['Age'].isNotNull())
                       .select('Pclass')
                       .distinct()
                       .collect())
p_class_count_non_null_df = (explore_age_df
                             .where(explore_age_df['Age'].isNotNull())
                             .groupBy('Pclass')
                             .count()
                             .select('count')
                             .collect())

data = [
    go.Bar(
        x= p_class_df,
        y= p_class_count_df
    ),
    go.Bar(
        x= p_class_non_null_df,
        y= p_class_count_non_null_df
    )
]

url = py.plot(data, filename='p-class-count', yTitle='# of passengers', xTitle='Passenger Class', title='Share of Passengers per class')

###### Visualization of Above
From this point, we can see that the passenger class of the records where `Age = null` is approximately that of the records where `Age` has a value.
<img src="http://i.imgur.com/6I3My9Cl.png" alt="Drawing" />

###### Analyzing a Dataframe of all 'null' Age datapoints.
You can see above that I was interested in the statistics of data points that did not possess any value for age. It turns out that... unfortunately, there is hardly any difference. At this point, I'm just going to say I'm bored and want to start doing some modeling. Not the best idea but let's try it and see what level of accuracy we get.